In [ ]:
!pip install trl
!pip install -U bitsandbytes
!pip install datasets
!pip install unbabel-comet


In [ ]:

import transformers
from transformers import (AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          AutoTokenizer,
                          TrainingArguments
                          )
from peft import LoraConfig,PeftModel
from trl import SFTTrainer
import torch

In [ ]:
#parameters
quant_config = BitsAndBytesConfig(use_4bit=True,
                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                  bnb_4bit_quant_type='nf4')
model_name = "NousResearch/Llama-2-7b-hf"


## Load pretrained

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=quant_config,
                                             device_map='auto')

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

#### Training arguments

## Data


In [ ]:
!wget https://s3.amazonaws.com/web-language-models/paracrawl/bonus/en-ru.txt.gz

--2025-03-23 10:31:49--  https://s3.amazonaws.com/web-language-models/paracrawl/bonus/en-ru.txt.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.64.46, 52.216.77.70, 52.217.116.192, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.64.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 667708245 (637M) [text/plain]
Saving to: ‘en-ru.txt.gz’

en-ru.txt.gz        100%[===================>] 636.78M  40.6MB/s    in 15s     

2025-03-23 10:32:05 (41.3 MB/s) - ‘en-ru.txt.gz’ saved [667708245/667708245]



In [ ]:
from datasets import load_dataset
#dataset is too big, load in streaming
dataset = load_dataset("text", data_files="en-ru.txt.gz",streaming=True)
SPLIT_TOKEN = "\t"

In [ ]:
# dataset_monolingual_ru = load_dataset("oscar-corpus/OSCAR-2201",
#                         language="ru",
#                         streaming=True, # optional
#                         split="train",
#                                       ) # optional
# dataset_monolingual_en = load_dataset("oscar-corpus/OSCAR-2201",
#                         language="en",
#                         streaming=True, # optional
#                         split="train") # optional



In [ ]:
# dataset_monolingual_ru.column_names

['id', 'text', 'meta']

In [ ]:
# from datasets import concatenate_datasets
# dataset_monolingual = concatenate_datasets([dataset_monolingual_ru, dataset_monolingual_en]).shuffle(seed=3)
#PROBLEM : the shuffle() operation is not really useful since it uses a buffer, which wont mix the 2 languages in the datasets.

## Prompt creation and preprocessing

In [ ]:
# Separate English and Russian

# PROMPT_TEMPLATE = """ ### Translate this sentence from {source} to {dest}, Source:
#  [source sentence]
#  ### Target:
#  """
PROMPT_TEMPLATE = """Translate this from {source} to {dest}.
{source}: {source_sentence}
 {dest}:"""
model_zero_shot = model
tokenizer_zero_shot = tokenizer

def zero_shot_eval_en_ru(prompt,ref):
  input_ids = tokenizer_zero_shot(prompt).input_ids # Return PyTorch tensors
  ref_toks = tokenizer_zero_shot(ref).input_ids

  max_len = len(ref_toks)
  len_prompt = len(input_ids)
  # input_ids = input_ids.to(model_zero_shot.device) # Move to the device the model is on
  input_ids = torch.tensor([input_ids], device=model_zero_shot.device)

  output_tokens = model_zero_shot.generate(input_ids,
                                            max_new_tokens=max_len)
                                            # eos_token_id is an attribute of the tokenizer
  output = tokenizer_zero_shot.decode(output_tokens[0][len_prompt:])
  return output

def build_en_to_ru(datapoint):
  text = datapoint["text"]
  en, ru = text.split(SPLIT_TOKEN)
  prompt = PROMPT_TEMPLATE.format(source="English", dest="Russian", source_sentence=en)
  mt = zero_shot_eval_en_ru(prompt,ru)

  return {"src": en, "ref": ru, "mt": mt}


ds_small = dataset["train"].take(20).map(build_en_to_ru)


## Zero-shot : COMET evaluationn

In [ ]:

from comet import download_model, load_from_checkpoint

comet_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_path)



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.40k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
comet_model = comet_model.to("cuda")
scores = []
for datapoint in ds_small.batch(8):
  # print(datapoint)
  scores.append(comet_model.predict([datapoint],batch_size=8))


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  4.09it/s]
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO

In [ ]:
print(list(map(lambda x: x[0],scores)))

[[0.5936281085014343], [0.5273339152336121], [0.6261472105979919]]
